<a href="https://colab.research.google.com/github/sindyalvinaa/PCVK_Genap_2023/blob/main/UAS_M_FaiqPrasetya_SindyAlvinaKhoiriyah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tugas Akhir PCVK

Anggota Kelompok:

*   Muhammad Faiq Prasetya - (2041720239)
*   Sindy Alvina Khoiriyah - (2041720003)



In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2 as cv
from google.colab.patches import cv2_imshow
import tables
import glob
from random import shuffle

In [4]:
directory = '/content/drive/MyDrive/PCVK/Images/project_uas/fish'

## Main Code

### Pre-processing

In [5]:
shuffle_data = True  # Acak alamat sebelum disimpan
hdf5_path = '/content/drive/MyDrive/PCVK/Images/project_uas/dataset.hdf5'
train_path = '/content/drive/MyDrive/PCVK/Images/project_uas/fish/*/*.jpg'
data_order = 'tf' # Pustaka berbeda menggunakan pengurutan data berbeda `tf` untuk aliran tensor.

# Baca alamat dan label dari folder 'Dataset'
addrs = glob.glob(train_path) # Menyimpan path dataset
labels = [0 if 'non-fresh' in addr else 1 for addr in addrs]
# Pengganti value 0 = non-fresh, 1 = fresh
# label = ['Tidak Segar' jika 'tidak segar' di addr lain 'Segar' untuk addr di addrs]
# Tidak berfungsi, one-hot encoding cuma bisa numeri


print(len(addrs)) # Pengecekan address
print(len(labels)) # Pengecekan label

# Mengacak data
if shuffle_data:
    c = list(zip(addrs, labels))
    shuffle(c)
    addrs, labels = zip(*c)

# Bagi data menjadi 60% train, 20% validasi, dan 20% test
train_addrs = addrs[0:int(0.6*len(addrs))]
train_labels = labels[0:int(0.6*len(labels))]

val_addrs = addrs[int(0.6*len(addrs)):int(0.8*len(addrs))]
val_labels = labels[int(0.6*len(addrs)):int(0.8*len(addrs))]

test_addrs = addrs[int(0.8*len(addrs)):]
test_labels = labels[int(0.8*len(labels)):]


print('train size:', len(train_addrs))
print('val size:', len(val_addrs))
print('test size:', len(test_addrs))

data_order = 'tf'  # 'th' untuk Theano, 'tf' untuk Tensorflow
img_dtype = tables.UInt8Atom()  # dtype di mana gambar akan disimpan

# periksa urutan data dan pilih bentuk data yang tepat untuk menyimpan gambar
if data_order == 'th':
    data_shape = (0, 3, 128, 128)
elif data_order == 'tf':
    data_shape = (0, 128, 128, 3)

# buka file hdf5 dan buat array
hdf5_file = tables.open_file(hdf5_path, mode = 'w')
try:
    train_storage = hdf5_file.create_earray(hdf5_file.root, 'train_img', img_dtype, shape=data_shape)
    val_storage = hdf5_file.create_earray(hdf5_file.root, 'val_img', img_dtype, shape=data_shape)
    test_storage = hdf5_file.create_earray(hdf5_file.root, 'test_img', img_dtype, shape=data_shape)

    mean_storage = hdf5_file.create_earray(hdf5_file.root, 'train_mean', img_dtype, shape=data_shape)

    # buat array label dan salin data label di dalamnya
    hdf5_file.create_array(hdf5_file.root, 'train_labels', train_labels)
    hdf5_file.create_array(hdf5_file.root, 'val_labels', val_labels)
    hdf5_file.create_array(hdf5_file.root, 'test_labels', test_labels)

    # array numpy untuk menyimpan rata-rata gambar
    mean = np.zeros(data_shape[1:], np.float32)

    # perulangan alamat
    for i in range(len(train_addrs)):
        # cetak berapa banyak gambar yang disimpan setiap 10 gambar
        if i % 10 == 0 and i > 1:
            print('Train data: {}/{}'.format(i, len(train_addrs)))

        # baca gambar dan ubah ukurannya menjadi (128, 128)
        # cv2 memuat gambar sebagai BGR, mengubahnya menjadi RGB
        addr = train_addrs[i]
        #cetak(addr)
        img = cv.imread(addr)
        img = cv.resize(img, (128, 128), interpolation = cv.INTER_CUBIC)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

        # Tambahkan pra-pemrosesan gambar

        # jika urutan data adalah Theano, urutan sumbu harus berubah
        if data_order == 'th':
            img = np.rollaxis(img, 2)

        # simpan gambar dan hitung mean
        train_storage.append(img[None])
        mean += img / float(len(train_labels))

    # mengulang alamat validasi
    for i in range(len(val_addrs)):
        # cetak berapa banyak gambar yang disimpan setiap 10 gambar
        if i % 10 == 0 and i > 1:
            print ('Validation data: {}/{}'.format(i, len(val_addrs)))

        # baca gambar dan ubah ukurannya menjadi (128, 128)
        # cv2 memuat gambar sebagai BGR, mengubahnya menjadi RGB
        addr = val_addrs[i]
        img = cv.imread(addr)
        img = cv.resize(img, (128, 128), interpolation = cv.INTER_CUBIC)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

        # tambahkan pra-pemrosesan gambar

        # jika urutan data adalah Theano, urutan sumbu harus berubah
        if data_order == 'th':
            img = np.rollaxis(img, 2)

        # simpan gambar
        val_storage.append(img[None])

    # mengulang alamat pengujian
    for i in range(len(test_addrs)):
        # cetak berapa banyak gambar yang disimpan setiap 10 gambar
        if i % 10 == 0 and i > 1:
            print ('Test data: {}/{}'.format(i, len(test_addrs)))

        # baca gambar dan ubah ukurannya menjadi (128, 128)
        # cv2 memuat gambar sebagai BGR, mengubahnya menjadi RGB
        addr = test_addrs[i]
        #cetak(addr)
        img = cv.imread(addr)
        img = cv.resize(img, (128, 128), interpolation = cv.INTER_CUBIC)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

        # tambahkan pra-pemrosesan gambar

        # jika urutan data adalah Theano, urutan sumbu harus berubah
        if data_order == 'th':
            img = np.rollaxis(img, 2)

        # simpan gambar
        test_storage.append(img[None])

    # simpan mean dan tutup file hdf5
    mean_storage.append(mean[None])
    print('HDF5 Done')
finally:
    print('In Finally')
    hdf5_file.close()

40
40
train size: 24
val size: 8
test size: 8
Train data: 10/24
Train data: 20/24
HDF5 Done
In Finally


### Model Training Menggunakan CNN

In [6]:
from math import ceil

In [7]:
hdf5_path = '/content/drive/MyDrive/PCVK/Images/project_uas/dataset.hdf5'

# Mengontrol apakah rata-rata akan dikurangkan dari data sebelum melatih model.
# Jika bernilai True, maka rata-rata akan dikurangkan dari data.
subtract_mean = True
batch_size = 50 # Ukuran batch yang akan digunakan selama proses pelatihan.
nb_class = 2 # Jumlah kategori yang ada dalam dataset

In [8]:
hdf5_file = tables.open_file(hdf5_path, mode = 'r')

# kurangi rata-rata pelatihan
if subtract_mean:
    mm = hdf5_file.root.train_mean[0]
    mm = mm[np.newaxis, ...]

# Jumlah total sampel

# train data : Data gambar pelatihan
# train label : Dat label pelatihan

train_data = np.array(hdf5_file.root.train_img)
train_label = np.array(hdf5_file.root.train_labels)

# test data : Data gambar pengujian
# test label : Data label Pengujian

test_data = np.array(hdf5_file.root.test_img)
test_label = np.array(hdf5_file.root.test_labels)

# val data : Data gambar validasi
# val label : Data label validasi

val_data = np.array(hdf5_file.root.val_img)
val_label = np.array(hdf5_file.root.val_labels)

print('train data:',train_data.shape,' train_label',train_label.shape)
print('test_data:',test_data.shape,' test_label:',test_label.shape)
print('val_data:',val_data.shape,' val_label:',val_label.shape)

train data: (24, 128, 128, 3)  train_label (24,)
test_data: (8, 128, 128, 3)  test_label: (8,)
val_data: (8, 128, 128, 3)  val_label: (8,)


In [9]:
from keras.utils import np_utils

# one-hot encode label
num_classes = len(np.unique(train_label))
train_label = np_utils.to_categorical(train_label, num_classes)
test_label = np_utils.to_categorical(test_label, num_classes)
val_label = np_utils.to_categorical(val_label, num_classes)

# bentuk cetak set pelatihan
print('num_classes:', num_classes)

# nomor cetak gambar pelatihan, validasi, dan uji
print(train_label.shape, 'train samples')
print(test_label.shape, 'test samples')
print(val_label.shape, 'validation samples')

num_classes: 2
(24, 2) train samples
(8, 2) test samples
(8, 2) validation samples


In [10]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu',
                        input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='tanh'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='tanh'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      208       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        8256      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 64)       0

In [11]:
# 'categorical_crossentropy' digunakan ketika kita memiliki klasifikasi multikelas dengan label yang diencode secara one-hot.
# optimizer : memperbarui bobot model.
# 'rmsprop' : algoritma jaringan saraf konvolusional (CNN).

model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics=['accuracy'])

In [12]:
from keras.callbacks import ModelCheckpoint
# ModelCheckpoint : menyimpan bobot (weights) model saat pelatihan dengan mengevaluasi kinerja model pada set data validasi.

# latih modelnya

# Checkpointer : mengonfigurasi pengaturan penyimpanan bobot model terbaik selama pelatihan.
# Filepath : menentukan nama file untuk menyimpan bobot model terbaik.
# verbose=1 : mencetak pesan saat menyimpan bobot
# save_best_only=True : menyimpan hanya bobot terbaik berdasarkan kinerja pada set data validasi.

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1,
                               save_best_only=True)
hist = model.fit(train_data, train_label, batch_size=None, epochs=20,
          validation_data=(val_data, val_label),callbacks=[checkpointer],
          verbose=1, shuffle=True)

Epoch 1/20
1/1 [==============================] - ETA: 0s - loss: 2.1338 - accuracy: 0.4583
Epoch 1: val_loss improved from inf to 140.48198, saving model to model.weights.best.hdf5
1/1 [==============================] - 3s 3s/step - loss: 2.1338 - accuracy: 0.4583 - val_loss: 140.4820 - val_accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - ETA: 0s - loss: 130.2482 - accuracy: 0.5417
Epoch 2: val_loss improved from 140.48198 to 19.70655, saving model to model.weights.best.hdf5
1/1 [==============================] - 1s 721ms/step - loss: 130.2482 - accuracy: 0.5417 - val_loss: 19.7066 - val_accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - ETA: 0s - loss: 22.7588 - accuracy: 0.4583
Epoch 3: val_loss improved from 19.70655 to 2.86040, saving model to model.weights.best.hdf5
1/1 [==============================] - 1s 742ms/step - loss: 22.7588 - accuracy: 0.4583 - val_loss: 2.8604 - val_accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - ETA

In [13]:
model.load_weights('model.weights.best.hdf5')